Загружаем бибилиотеки

In [1]:
import torch
from PIL import Image
import torchvision.transforms as transforms
from torchvision import models

from io import BytesIO
import requests

Определим вычислительный процессор

In [2]:
#Задаем список для интерпретации ответа модели
models_class = ('Картон', 'Стекло', 'Металл', 'Бумага', 'Пластик')

In [3]:
use_gpu = torch.cuda.is_available()
device = 'cuda' if use_gpu else 'cpu'

Модель Google

In [4]:
def google(): # pretrained=True для tensorflow
    model = models.googlenet(weights=models.GoogLeNet_Weights.IMAGENET1K_V1)
    model.fc = torch.nn.Linear(1024, 5)
    return model

Загрузка модели

In [5]:
path_to_model = 'google_ready_model.pth'
model = google()
model.load_state_dict(torch.load(path_to_model, weights_only=True, map_location=torch.device(device)))
model = model.to(device)
#Включить режим распознания
model.eval();

Нормализация входного изображения

In [6]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
tf = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Resize([70, 70]),
        normalize
    ]
)

Опишем функцию для анализа изображения по ссылке

In [7]:
def classify(url):
    # URL изображения
    image_url = url
    # Скачать изображение
    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content)).convert('RGB')
    image = tf(image).float().unsqueeze(0)
    image = image.to(device)
    output = model(image)
    _, predicted_class = torch.max(output, 1)
    class_index = predicted_class.item()
    return models_class[class_index]

Выполним тест нашей модели

In [8]:
test_image_addres = 'https://glstatic.rg.ru/uploads/images/2020/07/17/2d92ffc2066abeb.jpg'
print(classify(test_image_addres))

Картон


Запустим ТГ-бота.</br>
Основная задача определит материал объекта на фото

In [9]:
import nest_asyncio
# нужно ТОЛЬКО для запуска в ноутбуке
nest_asyncio.apply()

BOT_TOKEN = "Токен вашего ТГ-бота"

from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, ContextTypes, MessageHandler, filters

async def start_handler(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    await update.message.reply_text(f'🤖 Я умею определять материал предмета по фотографии: {', '.join(models_class)}')

async def hello_handler(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    await update.message.reply_text(f'Привет привет, {update.effective_user.first_name}. Я живой.')

async def fallback_handler(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    await update.message.reply_text(f'Отправьте мне картинку 🙏 а я определю материал ✨\nПодробнее: /start')

async def image_handler(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    try:
        new_file = await update.message.effective_attachment[-1].get_file()
        category = classify(new_file.file_path)
        await update.message.reply_text(f'На фото "{category}"')
    except Exception as e:
        await update.message.reply_text(f'😮‍💨 Мне никогда не сравниться с Человеком. Насчет "кожанного мешка" я пошутил, почини меня "{e}"')

app = ApplicationBuilder().token(BOT_TOKEN).build()

app.add_handler(CommandHandler("start", start_handler))
app.add_handler(CommandHandler("hello", hello_handler))
app.add_handler(MessageHandler(~ filters.PHOTO, fallback_handler))
app.add_handler(MessageHandler(filters.PHOTO, image_handler))   

app.run_polling();

RuntimeError: Cannot close a running event loop